In [ ]:
import onnx
import torch

In [ ]:
device = torch.device("cpu")

In [ ]:
fake_input = (1, 3, 640, 640)
fake_data = torch.zeros(fake_input)

In [ ]:
pt_model_name = "models/face_detection_model.pt"
pt_model = torch.load(pt_model_name, map_location=device)['model'].float()

In [ ]:
onnx_model_name = pt_model_name.replace(".pt",".onnx")
torch.onnx.export(pt_model, 
                  fake_data, 
                  onnx_model_name, 
                  opset_version=12,
                  input_names=['images'],
                  output_names=['num_dets', 'bboxes', 'scores', 'labels'])

In [ ]:
import onnxruntime

In [ ]:
opt_session = onnxruntime.SessionOptions()
opt_session.enable_mem_pattern = True
opt_session.enable_cpu_mem_arena = True
opt_session.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_EXTENDED

In [ ]:
ort_session = onnxruntime.InferenceSession(onnx_model_name, providers=['CPUExecutionProvider'])

In [ ]:
model_inputs = ort_session.get_inputs()
input_names = [model_inputs[i].name for i in range(len(model_inputs))]
input_names

In [ ]:
model_output = ort_session.get_outputs()
output_names = [model_output[i].name for i in range(len(model_output))]
output_names

In [ ]:
ort_inputs = {input_names[0]: fake_data.numpy()}
ort_outputs = ort_session.run(output_names, ort_inputs)

print(ort_outputs)